# Computing Stokes Coefficients
This notebook shows how to compute normalized Stokes coefficients from generic mascon models and geodesyNETs.

We use the following formal definition for the description of the gravitational potential (Tricarico 2013, Yoder 1995):

$$
U(r, \theta, \phi) = \frac \mu r \sum_{l=0}^{l=\infty}\sum_{m=0}^{m=l}\left(\frac {r_0}r\right)P_{lm}(cos\theta) \cdot\\
\cdot \left( C_{lm}\cos m\phi + S_{lm} \sin m\phi \right)
$$

where the Stokes coefficients relate to the body density via the formulas:


and

$$
C_{lm} = \frac{(2-\delta_{m,0})}{M}\frac{(l-m)!}{(l+m)!} \int_V \rho \left(\frac{r}{r_0} \right)^l \cdot\\
\cdot P_{lm}(\cos\theta)\cos m\phi dV
$$

and

$$
S_{lm} = \frac{(2-\delta_{m,0})}{M}\frac{(l-m)!}{(l+m)!} \int_V \rho \left(\frac{r}{r_0} \right)^l \cdot\\
\cdot P_{lm}(\cos\theta)\sin m\phi dV
$$

We also use the normalization factor $N_{lm}$ defined as:

$$
N_{lm} = \sqrt{\frac{(l+m)!}{(2-\delta_{m,0})(2l+1)(l-m)!} }
$$

so that our normalized Stokes coefficients are $\left\{\tilde C_{m,l}, \tilde
S_{m,l}\right\} = \left\{C_{m,l}, S_{m,l}\right\} N_{lm}$


In [ ]:
# Import our module containing helper functions
import gravann

# Core imports
import numpy as np
import scipy
import pickle as pk
import os
import torchquad as tquad
tquad.set_log_level("WARNING")

# pytorch
from torch import nn
import torch

# plotting stuff
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
%matplotlib notebook

# Ensure that changes in imported module (gravann most importantly) are autoreloaded
%load_ext autoreload
%autoreload 2

# If possible enable CUDA
gravann.enableCUDA()
gravann.fixRandomSeeds()
device = os.environ["TORCH_DEVICE"]
print("Will use device ",device)

tquad.set_precision("double")

# Loading and visualizing a mascon model
For the purpose of this nottebbok we will notebook we will be using, as mascon model, the one of Eros. Note though
that the procedure will be valid in general for any mascon model, that is N masses placed in N points in space.

In [ ]:
target = "models/Bennu.mdl"
target = target.split(".")[0].split("/")[1]

# We load the ground truth (a mascon model of some body)
with open("mascons/"+target+".pk", "rb") as file:
    mascon_points, mascon_masses, mascon_name = pk.load(file)
    
# We keep the data structure as numpy arrays (for now). Later we will use torch.
mascon_points = mascon_points
mascon_masses = mascon_masses

# Print some information on the loaded ground truth 
# (non-dimensional units assumed. All mascon coordinates are thus in -1,1 and the mass is 1)
print("Name: ", mascon_name)
print("Number of mascons: ", len(mascon_points))
print("Total mass: ", sum(mascon_masses))

In [ ]:
# Here we visualize the loaded mascon model
gravann.plot_mascon(mascon_points, mascon_masses)

# Stokes Coefficients From Mascon

In [ ]:
# Units of Length (this is Eros) (depend on how the the mascon model was created .... )
# These numbers matter for the final values quantitatively (but not qualitatively)
L = {
    "Bennu.pk": 352.1486930549145,
    "Bennu_nu.pk": 352.1486930549145,
    "Churyumov-Gerasimenko.pk": 3126.6064453124995, 
    "Eros.pk" : 20413.864850997925,
    "Itokawa.pk": 350.438691675663,
    "Itokawa_nu.pk": 350.438691675663,
    "Torus.pk": 3126.6064453124995,
    "Hollow.pk": 3126.6064453124995,
    "Hollow_nu.pk": 3126.6064453124995,
    "Hollow2.pk": 3126.6064453124995,
    "Hollow2_nu.pk": 3126.6064453124995
}

R0 = {
    "Eros.pk" : 16000,
    "Itokawa.pk": 300,
    "Churyumov-Gerasimenko.pk": 4300,
    "Bennu.pk": 565
}
L = L[target + ".pk"]
R0 = R0[target + ".pk"] / L

In [ ]:
with open("stokes/stokes_77_"+target+"_gt.pk", "rb") as file:
    stokesC_gt, stokesS_gt = pk.load(file)

In [ ]:
# Let us generate all Stokes coefficients up to order 7 and degree 7 (a square model)
l=7
m=7
stokesC_gt, stokesS_gt = gravann.mascon2stokes(mascon_points, mascon_masses, R0, l, m) #-> this should go on torch?

In [ ]:
stokesC_gt

In [ ]:
stokesS_gt

# Stokes Coefficients from godesyNET

In [ ]:
# Integrand to compute the mass
def mass(x):
    return model(x)

## We load the neural density field for Eros

In [ ]:
# Encoding: direct encoding (i.e. feeding the network directly with the Cartesian coordinates in the unit hypercube)
# was found to work well in most cases. But more options are implemented in the module.
encoding = gravann.direct_encoding()

# The model is here a SIREN network (FFNN with sin non linearities and a final absolute value to predict the density)
model = gravann.init_network(encoding, n_neurons=100, model_type="siren", activation = gravann.AbsLayer())
# Uncomment to run on CPU
#model.load_state_dict(torch.load("models/eros.mdl", map_location=torch.device('cpu')))
model.load_state_dict(torch.load("models/"+target+".mdl"))

... and we initialize the quadrature object and the associated Legendre polynomials

In [ ]:
# We construct the vecotrized Legendre associated polynomials
P = gravann.legendre_factory_torch(n = 16)
# Declare an integrator
quad = tquad.Boole()

... we compute the Asteroid mass from the model (this corresponds to 1/c)

In [ ]:
# Compute the function integral
N = 60**3
M = quad.integrate(mass,dim=3,N=N,integration_domain = [[-1,1],[-1,1],[-1,1]])
print(M)
torch.cuda.empty_cache()

In [ ]:
stokesC_gann = np.zeros((8,8))
for l in range(8):
    for m in range(8):
        if m>l:
            continue
        stokesC_gann[l][m] = quad.integrate(lambda x, l=l, m=m, P=P, model=model, R0=R0: gravann.Clm(x, model, l, m, R0, P), dim=3, N=N,integration_domain =[[-1,1],[-1,1],[-1,1]])
        stokesC_gann[l][m] = stokesC_gann[l][m]/M*gravann.constant_factors(l,m)
        print(f"C_{l}{m} {stokesC_gt[l][m]:2.3e} {stokesC_gann[l][m]:2.3e}")

In [ ]:
stokesS_gann = np.zeros((8,8))
for l in range(8):
    for m in range(8):
        if m>l:
            continue
        stokesS_gann[l][m] = quad.integrate(lambda x, l=l, m=m, P=P, model=model, R0=R0: gravann.Slm(x, model, l, m, R0, P), dim=3, N=N,integration_domain = [[-1,1],[-1,1],[-1,1]])
        stokesS_gann[l][m] = stokesS_gann[l][m]/M*gravann.constant_factors(l,m)
        print(f"C_{l}{m} {stokesS_gt[l][m]:2.3e} {stokesS_gann[l][m]:2.3e}")

In [ ]:
# Here we compute the sorted indexes corresponding to the largest values
idxs = np.dstack(np.unravel_index(np.argsort(-np.abs(stokesC_gt.ravel())), (8, 8)))[0]
print(f"Harmonics \tGround Truth \tGeodesyNET")
abs_errs_gann = []
rel_errs_gann = []

N_largest = 16

for n,pos in enumerate(idxs):
    if n == 0:
        continue
    if n == N_largest+1:
        break
    abs_errs_gann.append(abs(stokesC_gt[l][m] - stokesC_gann[l][m]))
    if(stokesC_gt[l][m] > 0):
        rel_errs_gann.append(abs(stokesC_gt[l][m] - stokesC_gann[l][m]) / stokesC_gt[l][m])
    l,m = pos
    print(f"C_{l}{m}, \t\t{stokesC_gt[l][m]:2.3e}, \t{stokesC_gann[l][m]:2.3e}")

print("geodesyNet Mean Abs. Err. =",np.mean(abs_errs_gann))
print("geodesyNet Mean Rel. Err. =",np.mean(rel_errs_gann))

In [ ]:
with open("stokes/stokes_77_"+target+"_gt.pk", "wb") as file:
    pk.dump((stokesC_gt, stokesS_gt), file)
with open("stokes/stokes_77_"+target+"_gann.pk", "wb") as file:
    pk.dump((stokesC_gann, stokesS_gann), file)